# Data Pre-processing

In [1]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv
# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
#Keras
import keras

Using TensorFlow backend.


In [0]:
import os
os.chdir("/content/drive/My Drive/228 project")

In [0]:
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
    header += f' mfcc{i}'
header += ' label'
header = header.split()

In [0]:
file = open('data.csv', 'w', newline='')
with file:
    writer = csv.writer(file)
    writer.writerow(header)
genres = os.listdir('./data')
for g in genres:
    for filename in os.listdir(f'./data/{g}'):
        name = f'./data/{g}/{filename}'
        print(name)
        
        y, sr = librosa.load(name)
        if(y.shape==(0,)):
          continue
        chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
        rmse = librosa.feature.rmse(y=y)
        spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
        spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
        rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
        zcr = librosa.feature.zero_crossing_rate(y)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        to_append = f'{filename} {np.mean(chroma_stft)} {np.mean(rmse)} {np.mean(spec_cent)} {np.mean(spec_bw)} {np.mean(rolloff)} {np.mean(zcr)}'    
        for e in mfcc:
            to_append += f' {np.mean(e)}'
        to_append += f' {g}'
        file = open('data.csv', 'a', newline='')
        with file:
            writer = csv.writer(file)
            writer.writerow(to_append.split())

In [0]:
data = pd.read_csv('new_data.csv')
data.head()

,filename,chroma_stft,rmse,spectral_centroid,spectral_bandwidth,rolloff,zero_crossing_rate,mfcc1,mfcc2,mfcc3,mfcc4,mfcc5,mfcc6,mfcc7,mfcc8,mfcc9,mfcc10,mfcc11,mfcc12,mfcc13,mfcc14,mfcc15,mfcc16,mfcc17,mfcc18,mfcc19,mfcc20,label
0,5e737f128dbbfd000703a1d2_3_27_cough.wav,0.522417,0.008602,2400.727343,2403.151804,5449.795313,0.139141,-486.748681,91.216387,4.192424,32.965005,-15.359769,27.052251,-6.170369,26.450120,-19.036352,15.613444,-5.508682,2.377509,6.950467,-1.473155,3.699778,-3.582184,6.878875,-3.985461,13.388008,-1.305689,pos_coughs
1,5e73b572cdc91d00083adc2e_3_27_cough.wav,0.348163,0.042541,2372.172967,2365.872150,4924.826168,0.137352,-364.272225,56.256711,4.490530,-13.735075,-11.417273,-11.267523,-9.017711,-10.873617,-12.118222,6.395580,1.452140,-3.985150,-3.975474,-2.633786,-8.009343,-6.574133,-4.668342,-2.972019,-3.152838,-3.154670,pos_coughs
2,5e73e0b9136acf00076f3294_3_27_cough.wav,0.523026,0.002268,2484.659211,2587.814267,5459.705395,0.114897,-615.064859,87.174039,4.612835,19.087210,6.235609,-2.844328,-8.259181,-6.340232,-5.483833,-2.213565,-7.907340,-7.462972,-13.542742,-10.122244,-9.192004,-8.111969,-2.951925,-0.967243,-5.502024,-2.266704,pos_coughs
3,5e748ce23fa893000702c84e_3_27_cough.wav,0.278881,0.000335,1719.142415,1438.114625,3368.321142,0.070156,-828.320780,35.502986,-2.645630,13.243908,6.860167,8.830194,4.249498,6.414262,0.246673,0.429023,-2.582951,-2.071157,-6.083559,-1.538721,-3.643860,-1.943024,-3.408480,3.326058,-1.036492,-1.380108,pos_coughs
4,5e77679f77dc8b00089e043a_3_27_cough.wav,0.378830,0.040679,1545.243736,1810.138382,3065.059204,0.077965,-422.530788,97.565362,11.227715,-1.284099,-7.051770,-17.845338,-14.666864,-10.014249,-12.494203,-13.521295,-13.508731,-10.048553,-14.073579,-9.971774,-7.768723,-9.567505,-10.585834,-7.540215,-6.640031,-5.847833,pos_coughs


In [0]:
data['label'].value_counts()

neg_coughs    1239
pos_coughs      82
Name: label, dtype: int64

In [0]:
## process data 
z=[]
for i in range(len(data)):
  if(data['chroma_stft'][i]==0.0):
    z.append(i)
data=data.drop(z)
data=data.to_csv("./new_data.csv",index=0)


# Load Data

In [4]:
data = pd.read_csv('./new_data.csv')
data['label'].value_counts()

# Dropping unneccesary columns
data = data.drop(['filename'],axis=1)
print(data.head())

data.label[data['label']=='pos_coughs']=1
data.label[data['label']=='neg_coughs']=0

Y = data['label']

# normalizing
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = float))
y=keras.utils.np_utils.to_categorical(Y, num_classes=2)

# spliting of dataset into train and test dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state =2)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

   chroma_stft      rmse  spectral_centroid  ...     mfcc19    mfcc20       label
0     0.522417  0.008602        2400.727343  ...  13.388008 -1.305689  pos_coughs
1     0.348163  0.042541        2372.172967  ...  -3.152838 -3.154670  pos_coughs
2     0.523026  0.002268        2484.659211  ...  -5.502024 -2.266704  pos_coughs
3     0.278881  0.000335        1719.142415  ...  -1.036492 -1.380108  pos_coughs
4     0.378830  0.040679        1545.243736  ...  -6.640031 -5.847833  pos_coughs

[5 rows x 27 columns]
(1056, 26)
(265, 26)
(1056, 2)
(265, 2)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


# CNN

In [6]:
## data reshape for CNN
X_train_conv=X_train.reshape([X_train.shape[0], X_train.shape[1],1,1 ])
X_train_conv.shape
X_test_conv=X_test.reshape([X_test.shape[0],X_test.shape[1],1,1])

## create a model 
model=keras.models.Sequential([
                               keras.layers.Conv2D(filters=256,kernel_size=(3,3),activation='relu',padding='same',input_shape=(26,1,1)),
                               keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'),
                               keras.layers.Conv2D(filters=128,kernel_size=(3,3),padding='same',activation='relu'), 
                               keras.layers.MaxPooling2D(pool_size=(2,2),padding='same'), 
                               keras.layers.Flatten(), 
                               keras.layers.Dense(units=128,activation='relu'), 
                               keras.layers.Dense(units=64,activation='relu'),
                               keras.layers.Dense(units=32, activation='relu'),
                               keras.layers.Dense(units=2,activation='softmax'),
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())           
history = model.fit(X_train_conv,
                    y_train,
                    epochs=20,
                    batch_size=10)

# calculate accuracy
test_loss, test_acc = model.evaluate(X_test_conv,y_test)
print('test_acc: ',test_acc)
# predictions
predictions = model.predict(X_test_conv)
print("negtive samples in test set ", sum(y_test==[1,0])[0])
print("positive samples in test set ",sum(y_test==[0,1])[0])
predictions[:,0]=predictions[:,0]>0.5
predictions[:,1]=predictions[:,1]>0.5

## 
neg_pos=0
neg_neg=0
pos_neg=0
pos_pos=0
for i in range(predictions.shape[0]):
  if((y_test[i]==[1,0])[0] & (predictions[i]==[0,1])[0]):
    neg_pos+=1
  if((y_test[i]==[1,0])[0] & (predictions[i]==[1,0])[0]):
    neg_neg+=1
  if((y_test[i]==[0,1])[0] & (predictions[i]==[1,0])[0]):
    pos_neg+=1
  if((y_test[i]==[0,1])[0] & (predictions[i]==[0,1])[0]):
    pos_pos+=1
print("negtive was diagnosed as positive", neg_pos)
print("negtive was diagnosed as negtive", neg_neg)
print("positive was diagnosed as negtive", pos_neg)
print("positive was diagnosed as positive", pos_pos)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 1, 256)        2560      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 1, 256)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 1, 128)        295040    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 1, 128)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 896)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 128)               114816    
_________________________________________________________________
dense_6 (Dense)              (None, 64)               

# Linear NN

In [7]:
# creating a model
model = keras.models.Sequential()
model.add(keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)))

model.add(keras.layers.Dense(64, activation='relu'))

model.add(keras.layers.Dense(32, activation='relu'))

model.add(keras.layers.Dense(2, activation='softmax'))

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())           
history = model.fit(X_train,
                    y_train,
                    epochs=40,
                    batch_size=100)

# calculate accuracy
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)
# predictions
predictions = model.predict(X_test)
print("negtive samples in test set ", sum(y_test==[1,0])[0])
print("positive samples in test set ",sum(y_test==[0,1])[0])
predictions[:,0]=predictions[:,0]>0.5
predictions[:,1]=predictions[:,1]>0.5

## 
neg_pos=0
neg_neg=0
pos_neg=0
pos_pos=0
for i in range(predictions.shape[0]):
  if((y_test[i]==[1,0])[0] & (predictions[i]==[0,1])[0]):
    neg_pos+=1
  if((y_test[i]==[1,0])[0] & (predictions[i]==[1,0])[0]):
    neg_neg+=1
  if((y_test[i]==[0,1])[0] & (predictions[i]==[1,0])[0]):
    pos_neg+=1
  if((y_test[i]==[0,1])[0] & (predictions[i]==[0,1])[0]):
    pos_pos+=1
print("negtive was diagnosed as positive", neg_pos)
print("negtive was diagnosed as negtive", neg_neg)
print("positive was diagnosed as negtive", pos_neg)
print("positive was diagnosed as positive", pos_pos)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 128)               3456      
_________________________________________________________________
dense_10 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_11 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_12 (Dense)             (None, 2)                 66        
Total params: 13,858
Trainable params: 13,858
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/40
1056/1056 [==============================] - 0s 129us/step - loss: 0.4153 - accuracy: 0.8873
Epoch 2/40
1056/1056 [==============================] - 0s 38us/step - loss: 0.2551 - accuracy: 0.9384
Epoch 3/40
1056/1056 [==============

# AutoEncoder

In [5]:
# # AutoEncoder
# # Read data
# data = pd.read_csv('data.csv')
# data = data.drop(['filename'],axis=1)

# y=[]
# for i in range(len(data)):
#     if(data['label'][i]=='pos_coughs'):
#       y.append(1)
#     else:
#       y.append(0)
# y=keras.utils.np_utils.to_categorical(y, num_classes=2)
# data =  data.drop(['label'],axis=1) 
# # Normalization
# scaler = StandardScaler()
# X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype = "float64"))
# AutoEncoder
model = keras.models.Sequential([
        keras.layers.InputLayer(input_shape=(X_train.shape[1],)),
        # Encoder
        keras.layers.Dense(20, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        # Decoder
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(2, activation='sigmoid'),
    ])
print(model.summary())
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
print(model.summary())           
hist = model.fit(X_train,
                    y_train,
                    epochs=150,
                    batch_size=100, shuffle=True)
# Confusion Matrix
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)
# predictions
predictions = model.predict(X_test)
print("negtive samples in test set ", sum(y_test==[1,0])[0])
print("positive samples in test set ",sum(y_test==[0,1])[0])
predictions[:,0]=predictions[:,0]>0.5
predictions[:,1]=predictions[:,1]>0.5

## 
neg_pos=0
neg_neg=0
pos_neg=0
pos_pos=0
for i in range(predictions.shape[0]):
  if((y_test[i]==[1,0])[0] & (predictions[i]==[0,1])[0]):
    neg_pos+=1
  if((y_test[i]==[1,0])[0] & (predictions[i]==[1,0])[0]):
    neg_neg+=1
  if((y_test[i]==[0,1])[0] & (predictions[i]==[1,0])[0]):
    pos_neg+=1
  if((y_test[i]==[0,1])[0] & (predictions[i]==[0,1])[0]):
    pos_pos+=1
print("negtive was diagnosed as positive", neg_pos)
print("negtive was diagnosed as negtive", neg_neg)
print("positive was diagnosed as negtive", pos_neg)
print("positive was diagnosed as positive", pos_pos)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                540       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 22        
Total params: 882
Trainable params: 882
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 20)                540       
_____________________________

# Random Forest

In [8]:
# data = pd.read_csv('data.csv')

# # Dropping unneccesary columns
# data = data.drop(['filename'],axis=1)

# # delete zero data
# zero_line = []
# for i in data.index:
#     if 0 in data.iloc[i].values:
#         zero_line.append(i)
# data.drop(zero_line).reset_index(inplace=True)

# # add new labels
# modify_label = []
# for i in data.index:
#     if data.iloc[i]['label'] == 'pos_coughs':
#         modify_label.append(1)
#     else:
#         modify_label.append(0)
# data['label_m'] = modify_label

# # normalizing
# scaler = StandardScaler()
# X = scaler.fit_transform(np.array(data.iloc[:, :-2], dtype = float))

# # spliting of dataset into train and test dataset
# X_train, X_test, y_train, y_test = train_test_split(X, data['label_m'], test_size=0.2)
y_train = y_train[:,1]
y_test  = y_test[:,1]

# Random Forest
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, \
    RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, mean_squared_error
from sklearn.model_selection import train_test_split, cross_validate, cross_val_score
import pylab as pl

rf = RandomForestClassifier(max_features=26, random_state=23)
rf.fit(X_train, y_train)
# scores_accuracy =  cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy')
# print(scores_accuracy.mean())

pred = rf.predict(X_test) #get the prediction on test (or validation) set

y_test1 = []

for i in y_test:
    if i == 1: y_test1.append('pos')
    else: y_test1.append('neg')
pred1 = []
for i in pred:
    if i == 1: pred1.append('pos')
    else: pred1.append('neg')
m = confusion_matrix(y_test, rf.predict(X_test)).T
cm = pd.DataFrame(m, columns=['neg', 'pos'], index=['neg', 'pos'])
cm.index.name = 'Predicted'
cm.columns.name = 'True'
print('Accuracy of negative predicted as negative: ', m[0][0]/(m[1][0] + m[0][0]))
print('Accuracy of positive predicted as positive: ', m[1][1]/(m[1][1] + m[0][1]))
cm

Accuracy of negative predicted as negative:  1.0
Accuracy of positive predicted as positive:  1.0


True,neg,pos
Predicted,,
neg,248,0
pos,0,17


# Boosting

In [9]:
# Boosting
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, BaggingRegressor, \
    RandomForestRegressor, GradientBoostingRegressor, GradientBoostingClassifier

bt = GradientBoostingClassifier(n_estimators=500, learning_rate=0.1, random_state=23)
bt.fit(X_train, y_train)

m=confusion_matrix(y_test, bt.predict(X_test)).T
cmcl1 = pd.DataFrame(m, index=['neg', 'pos'], columns=['neg', 'pos'])
cmcl1.index.name = 'Predicted'
cmcl1.columns.name = 'True'
print('Accuracy of negative predicted as negative: ', m[0][0]/(m[1][0] + m[0][0]))
print('Accuracy of positive predicted as positive: ', m[1][1]/(m[1][1] + m[0][1]))
cmcl1

Accuracy of negative predicted as negative:  1.0
Accuracy of positive predicted as positive:  1.0


True,neg,pos
Predicted,,
neg,248,0
pos,0,17
